In [37]:
import mirdata
import numpy as np
import os
import matplotlib.pyplot as plt
from lib.utils import smooth_pitch_curve, expand_zero_regions, interpolate_below_length, pitch_seq_to_cents, subsample_series

In [38]:
data_home = 'mirdatasets/Saraga'
dataset_name = 'saraga_carnatic'

In [39]:
dataset = mirdata.initialize(data_home=data_home, dataset_name=dataset_name)

In [40]:
#dataset.download(force_overwrite=True)

In [41]:
data = dataset.load_tracks()

In [42]:
pitch_dir = 'pitch'

In [49]:
for track_id, track in data.items():
    tonic = track.tonic
    pitch = track.pitch
    tempo = track.tempo

    if not track.metadata['raaga']:
        continue
        
    if track.metadata['raaga'][0]['name'] != 'Bhairavi':
        continue
    
    if not pitch or not tempo:
        continue
    
    beat = tempo['tempo_bpm'] / 60
    
    frequencies = pitch.frequencies
    times = pitch.times
    timestep = times[3] - times[2]

    frequencies = expand_zero_regions(frequencies, round(0.02 / timestep))
    frequencies = interpolate_below_length(frequencies, 0, (100 * 0.001 / timestep))
    cents = pitch_seq_to_cents(frequencies, tonic=tonic)
    time, cents = subsample_series(times, cents, 0.5)
    
    time_beat = time/beat
    try:
        cents = smooth_pitch_curve(time_beat, cents, smoothing_factor=0.4)
    except:
        continue

    np.save(f'{pitch_dir}/{track_id}', cents)